In [1]:
import os
import sys
import mc3
import pickle
import starry
import shutil
import subprocess
import progressbar
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

In [2]:
# Taurex imports
import taurex
from taurex import chemistry
from taurex import planet
from taurex import stellar
from taurex import model
from taurex import pressure
from taurex import temperature
from taurex import cache
from taurex import contributions
from taurex import optimizer
# This import is explicit because it's not included in taurex.temperature. Bug?
from taurex.data.profiles.temperature.temparray import TemperatureArray

# Taurex is a bit...talkative
import taurex.log
taurex.log.disableLogging()



In [3]:
# Directory structure
__file__ = os.path.abspath('') # __file__ is not defined in jupyter

maindir    = os.path.dirname(os.path.realpath(__file__))
libdir     = os.path.join(maindir, 'lib2')
moddir     = os.path.join(libdir,  'modules')
ratedir    = os.path.join(moddir,  'rate')
transitdir = os.path.join(moddir, 'transit')

In [4]:
# Lib imports
sys.path.append(libdir)
print(sys.path)
from lib2 import constants
from lib2 import cf
from lib2 import atm
from lib2 import my_pca as pca
from lib2 import star_eigen as eigen
from lib2 import model
from lib2 import star_plots as plots
from lib2 import mkcfg
from lib2 import star_utils as utils
from lib2 import constants   as c
from lib2 import star_fitclass    as fc
from lib2 import taurexclass as trc

starry.config.quiet = True

['/Library/Frameworks/Python.framework/Versions/3.9/lib/python39.zip', '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9', '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/.venv/lib/python3.9/site-packages', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/.venv/lib/python3.9/site-packages/mc3/lib/', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/.venv/lib/python3.9/site-packages/mc3/lib', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/lib2']


In [5]:
print(sys.argv)
cfile = "first_star.cfg"
dimen = "2D"

# if __name__ == "__main__":
#     print(sys.argv)
#     if len(sys.argv) < 3:
#         print("ERROR: Call structure is run.py <mode> <configuration file>.")
#         sys.exit()
#     else:
#         mode  = sys.argv[1]
#         cfile = sys.argv[2]

#     if mode in ['2d', '2D']:
#         finding_A(cfile)
#     # elif mode in ['3d', '3D']:
#     #     # Read config to find location of output, load output,
#     #     # then read config again to get any changes from 2d run.
#     #     fit = fc.Fit()
#     #     fit.read_config(cfile)
#     #     fit = fc.load(outdir=fit.cfg.outdir)
#     #     fit.read_config(cfile)
#     #     # 3D mapping doesn't care about the degree of harmonics, so
#     #     # just use 1
#     #     star, planet, system = utils.initsystem(fit, 1)
#     #     map3d(fit, system)
#     else:
#         print("ERROR: Unrecognized mode. Options are <2d, 3d>.")

['/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '--f=/Users/a.j.devaux/Library/Jupyter/runtime/kernel-v39aaef98a9bde7f05027e8b5b8283054b2762ea66.json']


In [6]:
fit = fc.Fit()
print("Reading the configuration file.")
fit.read_config(cfile)
cfg = fit.cfg

print("Reading the data.")
fit.read_data()

lmax = cfg.twod.lmax

print("First star and planet objects.")
star = utils.initsystem(fit, lmax)

Reading the configuration file.
Reading the data.
First star and planet objects.


In [7]:
print("Finding Inclination of planet")
print(cfg.star.inc)

Finding Inclination of planet
72.0


In [8]:
print("Finding Rank of Design Matrix")
A = star.map.design_matrix(theta=np.linspace(0,360,1000)) # how do I decide the angular phase???
# print(A.eval())

nsamples = cfg.twod.nsamples

# print(nsamples, lmax)

R = np.empty((nsamples, lmax))

for k in range(nsamples):
    
    R[k] = [
        np.linalg.matrix_rank(A.eval()[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
    ]
    # print(R[k])

R = np.median(R, axis=0)   

print(f"The design matrix is {R} for lmax = {lmax}")


Finding Rank of Design Matrix
The design matrix is [3. 5. 5.] for lmax = 3


In [9]:
print("Finding Eigencurves")

# Uniform star object
star = utils.initsystem(fit, 1)

fit.sflux = star.map.flux(theta=fit.t.shape).eval()

print(f"The sflux is: {fit.sflux}")

print(f"The star map has flux {star.map.flux(theta=np.linspace(0,360,len(fit.t))).eval()}")

print(f"The time shape is {fit.t.shape}")

print(star.map.inc.eval())

# star.map.show(theta=np.linspace(0, 360, 50))

Finding Eigencurves
The sflux is: [1.]
The star map has flux [1. 1. 1. ... 1. 1. 1.]
The time shape is (15937,)
72.0


In [10]:
fit.maps = []

fit.maps.append(fc.Map())
m = fit.maps[0]
m.ncurves = cfg.twod.ncurves
m.lmax    = cfg.twod.lmax
# m.wlmid = fit.wlmid[i]

if not os.path.isdir(cfg.outdir):
    os.mkdir(cfg.outdir)

m.subdir = 'filt{}'.format(1)
outdir = os.path.join(cfg.outdir, m.subdir)

if not os.path.isdir(os.path.join(cfg.outdir, m.subdir)):
    os.mkdir(os.path.join(cfg.outdir, m.subdir))

In [11]:
# New planet object with updated lmax
star = utils.initsystem(fit, m.lmax)

In [12]:
# print("Running PCA to determine eigencurves.")
# m.eigeny_i, m.evalues_i, m.evectors_i, m.ecurves_i, m.lcs_i = \
# eigen.mkcurves(star, fit.t, m.lmax, fit.sflux,
#                 ncurves=m.ncurves, method=cfg.twod.pca)

In [13]:
# print("Running PCA to determine eigencurves.")
# m.eigeny_j, m.evalues_j, m.evectors_j, m.ecurves_j, m.lcs_j = \
# eigen.mkcurves(star, fit.t, m.lmax, fit.sflux,
#                 ncurves=m.ncurves, method=cfg.twod.pca)

In [14]:
# #The lcs are the same!
# print((m.lcs_i == m.lcs_j).all())

# #The eigen maps are not!
# print((m.eigeny_i == m.eigeny_j).all())

# #The eigen maps are not!
# print((m.ecurves_i == m.ecurves_j).all())

# #The eigen maps are not!
# print((m.evectors_i == m.evectors_j).all())

# #The eigen maps are not!
# print((m.evalues_i == m.evalues_j).all())

In [15]:
#NEED TO CHANGE TO PCA IN CFG FILE

print("Running PCA to determine eigencurves.")
m.eigeny, m.evalues, m.evectors, m.ecurves, m.lcs = \
eigen.mkcurves(star, fit.t, m.lmax, fit.sflux,
                ncurves=m.ncurves, method="pca")

m.eigeny2, m.evalues2, m.evectors2, m.ecurves2, m.lcs2 = \
eigen.mkcurves(star, fit.t, m.lmax, fit.sflux,
                ncurves=m.ncurves, method="pca")

Running PCA to determine eigencurves.
(30, 15937)
(30, 15937)


In [16]:
#The lcs are the same!
print((m.lcs == m.lcs2).all())

#The eigen maps are not!
print((m.eigeny == m.eigeny2).all())

#The eigen maps are not!
print((m.ecurves == m.ecurves2).all())

#The eigen maps are not!
print((m.evectors == m.evectors2).all())

#The eigen maps are not!
print((m.evalues == m.evalues2).all())

True
True
True
True
True


In [17]:
print(f"The shape of lcs is {m.lcs.shape}\n")

print(f"The shape of ecurves (proj) is {m.ecurves.shape}\n")

# print(f"The shape of lcs with tsvd is {m.lcs_i.shape}\n")

# print(f"The shape of ecurves (proj) with tsvd is {m.ecurves_i.shape}\n")

The shape of lcs is (30, 15937)

The shape of ecurves (proj) is (30, 15937)



In [18]:
print(m.evectors.shape)

(30, 30)


In [19]:
print("Finding design matrix & rank of lcs")
lcs_A_T = m.lcs.T

lcs_R_T = np.empty((nsamples, lmax*2))

for k in range(nsamples):
    
    lcs_R_T[k] = [
        np.linalg.matrix_rank(lcs_A_T[:, : (l + 1) ** 2]) for l in range(1, lmax*2 + 1)
    ]
    # print(R[k])

lcs_R_T = np.median(lcs_R_T, axis=0)   

print(f"The new rank with lcs.T is {lcs_R_T}")

# lcs_A_T_i = m.lcs_i.T

# lcs_R_T_i = np.empty((nsamples, lmax*2))

# for k in range(nsamples):
    
#     lcs_R_T_i[k] = [
#         np.linalg.matrix_rank(lcs_A_T_i[:, : (l + 1) ** 2]) for l in range(1, lmax*2 + 1)
#     ]
#     # print(R[k])

# lcs_R_T_i = np.median(lcs_R_T_i, axis=0)   

# print(f"The new rank with lcs.T with tsvd is {lcs_R_T_i}")

Finding design matrix & rank of lcs
The new rank with lcs.T is [2. 3. 5. 5. 5. 5.]


In [38]:
print("Finding design matrix & rank of ecurves")

evect_A_T = m.ecurves.T

evect_R_T = np.empty((nsamples, lmax))

for k in range(nsamples):
    
    evect_R_T[k] = [
        np.linalg.matrix_rank(evect_A_T[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
    ]
    # print(R[k])

print(evect_R_T.shape)

evect_R_T = np.median(evect_R_T, axis=0)   

print(f"The new rank with ecurves.T is {evect_R_T}")

# evect_A_T_i = m.ecurves_i.T

# evect_R_T_i = np.empty((nsamples, lmax))

# for k in range(nsamples):
    
#     evect_R_T_i[k] = [
#         np.linalg.matrix_rank(evect_A_T_i[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
#     ]
#     # print(R[k])

# evect_R_T_i = np.median(evect_R_T_i, axis=0)   

# print(f"The new rank with ecurves.T with tsvd is {evect_R_T_i}")

Finding design matrix & rank of ecurves
(100, 3)
The new rank with ecurves.T is [4. 4. 4.]


In [21]:
print("Finding design matrix & rank of eigeny")
print(f"It has shape {m.eigeny.shape}")

eigeny_A = m.eigeny #m.eigeny.design_matrix()

eig_R = np.empty((nsamples, lmax))

for k in range(nsamples):

    eig_R[k] = [
        np.linalg.matrix_rank(eigeny_A[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
    ]
    # print(R[k])

eig_R = np.median(eig_R, axis=0)

print(f"The new rank with eigeny is {eig_R}")

eigeny_A_T = m.eigeny.T #m.eigeny.design_matrix()

eig_R_T = np.empty((nsamples, lmax))

for k in range(nsamples):

    eig_R_T[k] = [
        np.linalg.matrix_rank(eigeny_A_T[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
    ]
    # print(R[k])

eig_R_T = np.median(eig_R_T, axis=0)

print(f"The new rank with eigeny.T is {eig_R_T}")

# eigeny_A_T_i = m.eigeny_i.T #m.eigeny.design_matrix()

# eig_R_T_i = np.empty((nsamples, lmax))

# for k in range(nsamples):

#     eig_R_T_i[k] = [
#         np.linalg.matrix_rank(eigeny_A_T_i[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
#     ]
# # print(R[k])

# eig_R_T_i = np.median(eig_R_T_i, axis=0)

# print(f"The new rank with eigeny.T with tsvd is {eig_R_T_i}")

Finding design matrix & rank of eigeny
It has shape (15, 16)
The new rank with eigeny is [ 4.  9. 11.]
The new rank with eigeny.T is [ 4.  8. 11.]


In [22]:
plots.lightcurves(fit.t, m.lcs, outdir, "lcs_light_curves")

In [23]:
# plots look like they are doubling up!

plots.emaps(star, m.eigeny, outdir, proj='rect')

In [27]:
eigen_flux = []
ncurves, ny = m.eigeny.shape

lmax = int(ny**0.5 - 1)
thet = np.linspace(0, 360, ny)

# ncols = np.int(np.sqrt(ncurves) // 1)
# nrows = np.int(ncurves // ncols + (ncurves % ncols != 0))
# npane = ncols * nrows

for j in range(ncurves):
    star.map[1:,:] = 0

    # xloc = j %  ncols
    # yloc = j // ncols
    
    yi = 1
    for l in range(1, lmax + 1):
        for mn in range(-l, l + 1):
            star.map[l, mn] = m.eigeny[j, yi]
            yi += 1

    print(f"The {j} index eigen map's flux is {star.map.flux(theta=thet).eval()}")
    eigen_flux.append(star.map.flux(theta=thet).eval())

The 0 index eigen map's flux is [ 2.60439549  2.4656796   2.07352286  1.49574607  0.83226421  0.19780226
 -0.29794381 -0.56926893 -0.56926893 -0.29794381  0.19780226  0.83226421
  1.49574607  2.07352286  2.4656796   2.60439549]
The 1 index eigen map's flux is [ 1.          1.6525521   2.19227202  2.52583727  2.59557139  2.38941673
  1.94301929  1.33356525  0.66643475  0.05698071 -0.38941673 -0.59557139
 -0.52583727 -0.19227202  0.3474479   1.        ]
The 2 index eigen map's flux is [1.61918332 1.41429219 0.93520708 0.49896808 0.39426755 0.69040834
 1.19144026 1.56582484 1.56582484 1.19144026 0.69040834 0.39426755
 0.49896808 0.93520708 1.41429219 1.61918332]
The 3 index eigen map's flux is [1.         0.53978926 0.38411782 0.6359995  1.12875444 1.53630757
 1.58896518 1.25188169 0.74811831 0.41103482 0.46369243 0.87124556
 1.3640005  1.61588218 1.46021074 1.        ]
The 4 index eigen map's flux is [1.26064866 1.26064866 1.26064866 1.26064866 1.26064866 1.26064866
 1.26064866 1.2606486

In [30]:
print((eigen_flux[6] == eigen_flux[7]).all())

True
